In [1]:
!git clone https://github.com/vwoloszyn/eHealth_KD_2021
!pip install flair
import sys
sys.path.append("./eHealth_KD_2021/scripts/")
from anntools import Collection
from pathlib import Path
from flair.data import Sentence, Corpus
import numpy as np


fatal: destination path 'eHealth_KD_2021' already exists and is not an empty directory.


In [2]:
c = Collection()
for fname in Path("./eHealth_KD_2021/2021/training/").rglob("*.txt"):
    c.load(fname)

In [3]:
#
def brat2flair(brat):
    out=[]
    for s in brat:
        sent=Sentence(s.text)
        for t in sent:
            for k in s.keyphrases:
                if t.text in k.text:
                    t.add_tag('ner', k.label)
        out.append(sent)
    return out

x = np.split(c, [int(.8 * len(c)), int(.9 * len(c))])
sentences_train=brat2flair(x[0])
sentences_dev=brat2flair(x[1])
sentences_test=brat2flair(x[2])
corpus: Corpus = Corpus(sentences_train, sentences_dev, sentences_test)

In [4]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
import torch
from torch.optim.lr_scheduler import OneCycleLR


# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

print(tag_dictionary)

# 4. initialize fine-tuneable transformer embeddings WITH document context
from flair.embeddings import TransformerWordEmbeddings

embeddings = TransformerWordEmbeddings(
    #model='distilbert-base-uncased',
    model='xlm-roberta-base',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

#tagger: SequenceTagger = SequenceTagger(hidden_size=256,
#                                        embeddings=embeddings,
#                                        tag_dictionary=tag_dictionary,
#                                        tag_type=tag_type,
#                                         use_crf=True)

from flair.models import SequenceTagger
tagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=torch.optim.AdamW)

# 7. start training
trainer.train('resources/taggers/ner-roberta-large',
             learning_rate=5.0e-5,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=5,
              scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.,
              )



Dictionary with 9 tags: <unk>, O, Concept, , Action, Reference, Predicate, <START>, <STOP>
2021-03-18 14:57:09,104 ----------------------------------------------------------------------------------------------------
2021-03-18 14:57:09,107 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_fea

{'dev_loss_history': [0.39788687229156494,
  0.37750276923179626,
  0.39883142709732056,
  0.41126543283462524,
  0.4335750341415405],
 'dev_score_history': [0.8214, 0.83, 0.8436, 0.8419, 0.8386],
 'test_score': 0.8231,
 'train_loss_history': [0.6213831873734792,
  0.46549285179159294,
  0.3840128321914623,
  0.28574789300289316,
  0.2638020537192157]}